In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

%matplotlib inline

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
#Combine train and test feature data
all_data = pd.concat((train.loc[:, "MSSubClass": "SaleCondition"],
                     test.loc[:, "MSSubClass": "SaleCondition"]))
all_data.head()

In [ ]:
#SalePrice is right-skewed. Log transform label data
train["SalePrice"] = np.log1p(train["SalePrice"])

In [ ]:
#Log transform skewed features

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])


In [ ]:
all_data = pd.get_dummies(all_data)

In [ ]:
all_data = all_data.fillna(all_data.median())

In [ ]:
#Process data for Sklearn algorithms
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

In [ ]:
#import algorithms and define function to calculate rmse
from sklearn.linear_model import Ridge, RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv=5))
    return(rmse)

In [ ]:
model = Ridge()
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

In [ ]:
#plot alpha value versus rmse
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot()
plt.xlabel("alpha")
plt.ylabel("rmse")


In [ ]:
cv_ridge.min()

In [ ]:
model_lasso = LassoCV(alphas=[1, 0.1, 0.001, 0.0005]).fit(X_train, y)
rmse_cv(model_lasso).mean()

In [ ]:
#Add xgboost model to linear regrassion with lasso
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y)
dtest = xgb.DMatrix(X_test)

In [ ]:
params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain, num_boost_round=500, early_stopping_rounds=100)

In [ ]:
model.loc[30:, ["test-rmse-mean", "train-rmse-mean"]].plot()

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {"n_estimators":[350, 375, 400, 425], "max_depth": [2, 3, 4]}

xgb = xgb.XGBRegressor(learning_rate=0.1)
grid_xgb = GridSearchCV(xgb, params)
grid_xgb.fit(X_train, y)

In [ ]:
grid_xgb.best_params_

In [ ]:
xgb_preds = np.expm1(grid_xgb.best_estimator_.predict(X_test))
lasso_preds = np.expm1(model_lasso.predict(X_test))

In [ ]:
preds = 0.7*lasso_preds + 0.3*xgb_preds

In [ ]:
xgb_solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
xgb_solution.head()

In [ ]:
xgb_solution.to_csv("xgb_lasso_sol2.csv", index=False)